<h1 style="text-align: center; font-family: Arial"> Two exercices to test python skills  </h1>

**Learning outcomes:**
- Download/Upload from Dropbox
- Edit cvs file using pandas
- Web scraping
- Ants Registration

In [1]:
#Display with Notebook's built-in graphics library
%matplotlib inline

In [2]:
import pathlib #offer classes representing filesystem paths with semantics appropriate for different operating systems
import dropbox #API for downloading/uploading from/to a dropbox account or case
from dropbox.exceptions import AuthError

import pandas as pd #open source data analysis and manipulation tool
import random #generate random numbers
from datetime import date,timedelta #manipulate dates (year, month, day)

import nibabel as nib #supports an ever growing collection of neuroimaging file formats
import requests #standard for making HTTP requests 
from bs4 import BeautifulSoup #library for parsing structured data

import os

import matplotlib.pyplot as plt

from ipywidgets import interact
import numpy as np
import SimpleITK as sitk
import cv2


#from helpers import * #.py file from a tutorial
import ants #AntsPy only used to visualize mask contour. Use ants from channel pnlbwh on terminal to get output files.
import SimpleITK as sitk #Visualize MRI images
import subprocess #Execute bash commands to use ANTS from PNLBWH

#import itk #to do the automatic itk-skullstripping

print(f'AntsPy version = {ants.__version__}')
print(f'SimpleITK version = {sitk.__version__}')

AntsPy version = 0.3.8
SimpleITK version = 2.1.1.1


In [3]:
#print working directory
BASE_DIR = os.path.dirname(os.path.abspath("__file__"))
print(f'project folder = {BASE_DIR}')

project folder = /home/bverreman/Downloads/coding-test-jupyter-notebook


# Exercice I: Anonymize a dataset on Dropbox

**1.** Set up the Dropbox application

In [10]:
DROPBOX_ACCESS_TOKEN = 'sl.BfXIXkcZOAVQeXYYNRTb5uar0dXJD4c0tk8_4QZITkJS7n7OAIw0Jt7GJ3OmHULLQXdyH57YJlx0ZajGm_8KJ92eiyOhrWlJ82RxyPd_5YrPaixni1Le0DkB5HMTuKjGS6YSZzPr'
#Security danger : unlimited access to my whole Dropbox account using this access token (use OAuth 2 instead to give authorization each time)
#Access token only valid for 24 hours

**2.** Connect to the Dropbox API

In [11]:
def dropbox_connect():
    """Create a connection to Dropbox."""

    try:
        dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN)
    except AuthError as e:
        print('Error connecting to Dropbox with access token: ' + str(e))
    return dbx

**3.** Download the csv file from Dropbox

In [12]:
def dropbox_download_file(dropbox_file_path, local_file_path):
    """Download a file from Dropbox to the local machine."""

    try:
        dbx = dropbox_connect()

        with open(local_file_path, 'wb') as f:
            metadata, result = dbx.files_download(path=dropbox_file_path)
            f.write(result.content)
    except Exception as e:
        print('Error downloading file from Dropbox: ' + str(e))


In [13]:
#Download the four files from Dropbox repository
local_file_path="enroll_data.csv"
dropbox_file_path="/recruitment_project/"+local_file_path
dropbox_download_file(dropbox_file_path, local_file_path)

local_file_path="atlas-T1w.nii.gz"
dropbox_file_path="/recruitment_project/"+local_file_path
dropbox_download_file(dropbox_file_path, local_file_path)

local_file_path="atlas-integer-labels.nii.gz"
dropbox_file_path="/recruitment_project/"+local_file_path
dropbox_download_file(dropbox_file_path, local_file_path)

local_file_path="given-T1w.nii.gz"
dropbox_file_path="/recruitment_project/"+local_file_path
dropbox_download_file(dropbox_file_path, local_file_path)

**4.** Read csv file and anonymize the data

In [14]:
def anonymization(f,f_anon,f_offset):
    """Create two csv files in working directory : 
    f_anon: anonymized data from the source file f (disguised date of consent and birth date replaced by age at date of consent)
    f_offset: offset data used to deanonymize
        
    Args:
        f (str): The name to the local source file.
        f_anon (str): The name of the anonimized file.
        f_offset (str): The name of the offset file.

    Example:
        dropbox_upload_file('enroll_data.csv', 'enroll_data_anon_BV.csv', 'enroll_data_offset_BV.csv')
    """
    #Seed used for the reproductibility of the results and debugging. Should be removed to garantee anonymization.
    random.seed(1) 
    
    #Read enroll_data.csv
    with open(f, 'r', newline='') as originF: 
        lines=originF.readlines()
    
    #Get date of consent and birth date values
    df_anon = pd.DataFrame([x.split(',') for x in lines])
    df_anon = df_anon.drop([0])
    df_anon.rename(columns = {0:'site ID',1:'date of consent',2:'cohort',3:'birth date'}, inplace = True)
    
    consent=list(df_anon['date of consent'])
    consent_list=[x.split('/') for x in consent]
    
    birth=list(df_anon['birth date'])
    birth_list=[x.split('-') for x in birth]
    
    #Compute the new date of consent (with random offset)
    #Compute the age at the original date of consent
    new_consent=[]
    new_age=[]
    new_offset=[]
    for i in range(len(consent)):
        consent_date = date(year=int(consent_list[i][2]),month=int(consent_list[i][0]),day=int(consent_list[i][1]))
        
        birth_date = date(year=int(birth_list[i][0]),month=int(birth_list[i][1]),day=int(birth_list[i][2]))
        age=consent_date.year-birth_date.year
        if consent_date.month<birth_date.month:
            age+=1
        elif consent_date.month==birth_date.month:
            if consent_date.day<=birth_date.day:
                age+=1
        new_age.append(age)
            
        offset=random.randint(35065, 37065) #96years between 1924 and 2020: 25*366+71*365=35065 days exactly
        new_consent_date=consent_date-timedelta(days=offset) #days substraction
        
        new_date=str(new_consent_date.month)+'/'+str(new_consent_date.day)+'/'+str(new_consent_date.year)
        new_consent.append(new_date)
        
        new_offset.append(offset)
        
    #Update anonymization dataframe (df_anon) and create offset dataframe (df_offset)
    df_anon.drop('date of consent', inplace=True, axis=1)
    df_anon.insert (1, 'date of consent', new_consent)
    
    df_anon.drop('birth date', inplace=True, axis=1)
    df_anon.insert (3, 'age', new_age)
    print(df_anon)
    
    df_offset = pd.DataFrame(new_offset,columns=['offset'])
    print(df_offset)
        
    #Write the two csv files
    df_anon.to_csv(f_anon)
    df_offset.to_csv(f_offset)

In [15]:
f = "enroll_data.csv"
f_anon = "enroll_data_anon_BV.csv"
f_offset = "enroll_data_offset_BV.csv"
anonymization(f,f_anon,f_offset)

     site ID date of consent cohort  age
1        BWH       3/31/1923    CHR   31
2        BWH      10/23/1920    CHR   32
3        BWH        4/2/1919     HC   23
4        BWH        7/3/1919     HC   34
5        BWH       9/20/1919    CHR   35
...      ...             ...    ...  ...
7942     PNC       1/22/1920    CHR   21
7943     PNC      11/30/1924     HC   32
7944     PNC       1/18/1923    CHR   33
7945     PNC      11/29/1920    CHR   24
7946     PNC       9/26/1919    CHR   35

[7946 rows x 4 columns]
      offset
0      35340
1      36230
2      36800
3      36708
4      36629
...      ...
7941   36869
7942   35095
7943   35777
7944   36557
7945   36987

[7946 rows x 1 columns]


**5.** Upload the files to Dropbox

In [16]:
def dropbox_upload_file(local_path, local_file, dropbox_file_path):
    """Upload a file from the local machine to a path in the Dropbox app directory.

    Args:
        local_path (str): The path to the local file.
        local_file (str): The name of the local file.
        dropbox_file_path (str): The path to the file in the Dropbox app directory.

    Example:
        dropbox_upload_file('.', 'test.csv', '/stuff/test.csv')

    Returns:
        meta: The Dropbox file metadata.
    """

    try:
        dbx = dropbox_connect()

        local_file_path = pathlib.Path(local_path) / local_file

        with local_file_path.open("rb") as f:
            meta = dbx.files_upload(f.read(), dropbox_file_path, mode=dropbox.files.WriteMode("overwrite"))

            return meta
    except Exception as e:
        print('Error uploading file to Dropbox: ' + str(e))


In [17]:
local_file="enroll_data_anon_BV.csv"
dropbox_file_path="/recruitment_project/"+local_file
dropbox_upload_file("", local_file, dropbox_file_path)

local_file="enroll_data_offset_BV.csv"
dropbox_file_path="/recruitment_project/"+local_file
dropbox_upload_file("", local_file, dropbox_file_path);

# Exercice II: Ants Registration

**1.** Web scrapping: FreeSurfer Look up table

In [18]:
url = "https://surfer.nmr.mgh.harvard.edu/fswiki/FsTutorial/AnatomicalROI/FreeSurferColorLUT"

page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="content") #Find the 'div' with id=content
res=results.text #Remove html code to only keep text

with open("LookUpTable.txt", "w+") as f:
    f.write(res)
    f.seek(0) #put file pointer at the start
    lines=f.readlines()
with open("LookUpTable.txt", "w+") as f:
    for line in lines[:-1]:
        if not line.startswith('#') and not line.startswith('\n'):
            f.write(line)
    line=lines[-1]
    f.write(line.split('*')[0]) #remove comment on last line 
    f.seek(0) 
    lines=f.readlines()

#Create pandas dataframe
df = pd.DataFrame([x.split() for x in lines])
df.rename(columns = {0:'label',1:'name'}, inplace = True)
table=df[['label','name']]
table=table.astype({'label': 'int'})
#print(type(table.at[0,"label"]))
print(table)

      label                         name
0         0                      Unknown
1         1       Left-Cerebral-Exterior
2         2   Left-Cerebral-White-Matter
3         3         Left-Cerebral-Cortex
4         4       Left-Lateral-Ventricle
...     ...                          ...
1287  14171           wm_rh_S_suborbital
1288  14172          wm_rh_S_subparietal
1289  14173         wm_rh_S_temporal_inf
1290  14174         wm_rh_S_temporal_sup
1291  14175  wm_rh_S_temporal_transverse

[1292 rows x 2 columns]


**2.** Visual aspect of raw image, atlas and atlas integer labels

In [25]:
def show_info(img_sitk,img_name):
    """Show information of an MRI image

    Args:
        img_name: MRI image in working directory

    Example:
        show_info(img_sitk,"given-T1w.nii.gz")
    """
    print("\n"+"Image : "+img_name)
    
    pixel_type = img_sitk.GetPixelIDTypeAsString()
    origin = img_sitk.GetOrigin()
    dimensions = img_sitk.GetSize()
    spacing = img_sitk.GetSpacing()
    direction = img_sitk.GetDirection()

    info = {'Pixel Type' : pixel_type, 'Dimensions': dimensions, 'Spacing': spacing, 'Origin': origin,  'Direction' : direction}
    for k,v in info.items():
        print(f' {k} : {v}')

    img_sitk_arr = sitk.GetArrayFromImage(img_sitk)
    print(f'type = {type(img_sitk_arr)}')
    print(f'shape = {img_sitk_arr.shape}')

In [20]:
def show_image(img_sitk,orientation1='LAS',orientation2='AIR',cmap='gray') : #Orientation (X,Y,Z), into the direction: Right-Left/Inferior-Superior/Posterior-Anterior
    """Show an MRI image with slice bar using simpleITK and in two orientations

    Args:
        img_name: MRI image in working directory using simpleITK
        orientation1: orientation of the first image showed
        orientation2: orientation of the second image showed

    Example:
        show_image("given-T1w.nii.gz",'LAS','AIR')
    """
    def fn(SLICE):
        plt.figure(figsize=(7,7))
        plt.imshow(img_sitk_arr[SLICE, :, :], cmap=cmap)
    
    img_sitk_cp = sitk.DICOMOrient(img_sitk,orientation1)
    img_sitk_arr=sitk.GetArrayFromImage(img_sitk_cp)
    interact(fn, SLICE=(0, img_sitk_arr.shape[0]-1))

    img_sitk_cp = sitk.DICOMOrient(img_sitk,orientation2)
    img_sitk_arr=sitk.GetArrayFromImage(img_sitk_cp)
    interact(fn, SLICE=(0, img_sitk_arr.shape[0]-1))

In [21]:
def show_compare(img1_sitk,img2_sitk,orientation1='LAS',orientation2='AIR',cmap='nipy_spectral') : #Orientation (Z,Y,X), from the direction: Right-Left/Inferior-Superior/Posterior-Anterior
    """Show two same sized MRI images from working directory with slice bar using simpleITK and in two orientations

    Args:
        img1_sitk: First MRI image with simpleITK
        img2_sitk: Second MRI image with simpleITK (same size)
        orientation1: orientation of the first couple of image showed
        orientation2: orientation of the second couple of image showed
        cmap: color of cmap

    Example:
        show_compare(cropped_img_sitk,
        corrected_image_full_resolution, 
        orientation1='LAS',
        orientation2='AIR',
        cmap='gray')
    """    
    def fn(SLICE):
        fig, (ax1, ax2) = plt.subplots(1, 2, sharex='col', sharey='row', figsize=(10,10))

        ax1.set_title('Before', fontsize=15)
        ax1.imshow(img1_sitk_arr[SLICE, :, :], cmap=cmap)

        ax2.set_title('After', fontsize=15)
        ax2.imshow(img2_sitk_arr[SLICE, :, :], cmap=cmap)

        plt.tight_layout()
        
    img1_sitk_cp = sitk.DICOMOrient(img1_sitk,orientation1)
    img1_sitk_arr = sitk.GetArrayFromImage(img1_sitk_cp)
    
    img2_sitk_cp = sitk.DICOMOrient(img2_sitk,orientation1)
    img2_sitk_arr = sitk.GetArrayFromImage(img2_sitk_cp)
    
    assert img1_sitk_arr.shape == img2_sitk_arr.shape
    
    interact(fn, SLICE=(0, img1_sitk_arr.shape[0]-1))
    
    img1_sitk_cp = sitk.DICOMOrient(img1_sitk,orientation2)
    img1_sitk_arr = sitk.GetArrayFromImage(img1_sitk_cp)
    img2_sitk_cp = sitk.DICOMOrient(img2_sitk,orientation2)
    img2_sitk_arr = sitk.GetArrayFromImage(img2_sitk_cp)
    
    interact(fn, SLICE=(0, img1_sitk_arr.shape[0]-1))

In [22]:
def rescale_linear(array: np.ndarray, new_min: int, new_max: int):
    """Rescale an array linearly.
    
    Args:
        array: a array to rescale (type ndarray)
        new_min: new minimum value
        new_max: new maximum value
    
    Return:
        an array rescaled

    Example:
        rescale_linear(img_ants_arr,0,1)
    """
    minimum, maximum = np.min(array), np.max(array)
    m = (new_max - new_min) / (maximum - minimum)
    b = new_min - m * minimum
    return m * array + b

In [44]:
def show_superpose(img_name,mask_name,orientation1='SAL',orientation2='RIA',thickness: int = 1) : #Orientation (Z,Y,X) : Right-Left/Inferior-Superior/Posterior-Anterior
    """Superposes the outline of the mask on the corresponding MRI image 
    Apply a threshold on mask (integer labels) to make it binary
    Both images are from working directory 
    Also shows a slice bar, uses AntsPy and shows two orientations

    Args:
        img_name: MRI image name
        mask_name: MRI mask image name (same size)
        orientation1: orientation of the first couple of image showed
        orientation2: orientation of the second couple of image showed
        thickness: thickness of the outline in pixel (int)

    Example:
        show_superpose("corrected-T1w.nii.gz",
        label_to_corrected.nii.gz,
        orientation1='SAL',
        orientation2='RIA')
    """
    def fn(SLICE):
        arr_rgb = cv2.cvtColor(_arr[SLICE, :, :], cv2.COLOR_GRAY2RGB)
        contours, _ = cv2.findContours(_mask[SLICE, :, :], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        arr_with_contours = cv2.drawContours(arr_rgb, contours, -1, (0,1,0), thickness)

        plt.figure(figsize=(7,7))
        plt.imshow(arr_with_contours)
    
    img_ants = ants.image_read(BASE_DIR+"/"+img_name, reorient='IPL')
    mask_ants = ants.image_read(BASE_DIR+"/"+mask_name, reorient='IPL')
    mask_ants=ants.threshold_image(mask_ants, low_thresh=1,high_thresh=255)
    
    img_ants_arr=img_ants.numpy()
    mask_ants_arr=mask_ants.numpy()
    
    assert img_ants_arr.shape == mask_ants_arr.shape
    
    _arr = rescale_linear(img_ants_arr,0,1)
    _mask = rescale_linear(mask_ants_arr,0,1)
    _mask = _mask.astype(np.uint8)
    
    interact(fn, SLICE=(0, img_ants_arr.shape[0]-1))
    
    img_ants = ants.image_read(BASE_DIR+"/"+img_name, reorient='RSP')
    mask_ants = ants.image_read(BASE_DIR+"/"+mask_name, reorient='RSP')
    mask_ants=ants.threshold_image(mask_ants, low_thresh=1,high_thresh=255)
    
    img_ants_arr=img_ants.numpy()
    mask_ants_arr=mask_ants.numpy()
    
    _arr = rescale_linear(img_ants_arr,0,1)
    _mask = rescale_linear(mask_ants_arr,0,1)
    _mask = _mask.astype(np.uint8)
    
    interact(fn, SLICE=(0, img_ants_arr.shape[0]-1))

In [26]:
img_name="given-T1w.nii.gz"
img_sitk = sitk.ReadImage(BASE_DIR+"/"+img_name, sitk.sitkFloat32)
show_info(img_sitk,img_name)


Image : given-T1w.nii.gz
 Pixel Type : 32-bit float
 Dimensions : (176, 256, 256)
 Spacing : (0.9999160170555115, 0.9999999403953552, 0.9999999403953552)
 Origin : (-87.66697692871094, 111.55906677246094, -157.77081298828125)
 Direction : (1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
type = <class 'numpy.ndarray'>
shape = (256, 256, 176)


In [27]:
img_name="atlas-T1w.nii.gz"
img_sitk = sitk.ReadImage(BASE_DIR+"/"+img_name, sitk.sitkFloat32)
show_info(img_sitk,img_name)


Image : atlas-T1w.nii.gz
 Pixel Type : 32-bit float
 Dimensions : (182, 218, 182)
 Spacing : (1.0, 1.0, 1.0)
 Origin : (-90.0, 126.0, -72.0)
 Direction : (1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
type = <class 'numpy.ndarray'>
shape = (182, 218, 182)


In [28]:
img_sitk = sitk.ReadImage(BASE_DIR+"/"+"given-T1w.nii.gz", sitk.sitkFloat32)
show_image(img_sitk,'LAS','AIR')

interactive(children=(IntSlider(value=127, description='SLICE', max=255), Output()), _dom_classes=('widget-int…

interactive(children=(IntSlider(value=87, description='SLICE', max=175), Output()), _dom_classes=('widget-inte…

In [29]:
img_path=BASE_DIR+"/"+"atlas-T1w.nii.gz"
img_sitk = sitk.ReadImage(img_path, sitk.sitkFloat32)
show_image(img_sitk,'LAS','AIR')

interactive(children=(IntSlider(value=90, description='SLICE', max=181), Output()), _dom_classes=('widget-inte…

interactive(children=(IntSlider(value=90, description='SLICE', max=181), Output()), _dom_classes=('widget-inte…

In [30]:
img_path=BASE_DIR+"/"+"atlas-integer-labels.nii.gz"
img_sitk = sitk.ReadImage(img_path, sitk.sitkFloat32)
show_image(img_sitk,'LAS','AIR')

interactive(children=(IntSlider(value=90, description='SLICE', max=181), Output()), _dom_classes=('widget-inte…

interactive(children=(IntSlider(value=90, description='SLICE', max=181), Output()), _dom_classes=('widget-inte…

**Visual observations:**
- The raw image has a lot of white noise inside the white and the gray matter respectively
- The face on the raw image is masked to guarantee the anonymity
- The brains on both the raw image and the atlas are centered and aligned, but they are not the same relative size

**Recommendations for pre-processing**
- Crop the raw image to get a relative brain size indentical to the atlas
- Use Bias Field Correction on the raw image to reduce the noise both inside the white and the gray matter

**3.** Pre-processing before registration

**3.1** Cropping the raw image

In [31]:
def cropping(img_name,set1=(0,0,0),set2=(0,0,0),orientation1='LAS',orientation2='AIR'):
    """Crop an MRI image from working directory using simpleITK.
    Also show resulting image in two orientations.

    Args:
        img_name: MRI image name
        set1: number of pixels cropped in direction (Right,Posterior,Superior)
        set2: number of pixels cropped in direction (Left,Anterior,Inferior)
        orientation1: orientation of the first image showed
        orientation2: orientation of the second image showed

    Example:
        cropping("given-T1w.nii.gz",(5,20,70),(5,40,10))
    """
    img_sitk = sitk.ReadImage(BASE_DIR+"/"+img_name, sitk.sitkFloat32)
    img_sitk = sitk.Crop(img_sitk,set1,set2)

    img_sitk = sitk.DICOMOrient(img_sitk,orientation1)
    show_image(img_sitk,orientation1='LAS',orientation2='AIR',cmap='gray')  
    
    img_sitk = sitk.DICOMOrient(img_sitk,orientation2)
    show_image(img_sitk,orientation1='LAS',orientation2='AIR',cmap='gray') 
    
    return img_sitk
    

In [33]:
cropped_img_sitk=cropping("given-T1w.nii.gz",(5,20,70),(5,40,10))

interactive(children=(IntSlider(value=87, description='SLICE', max=175), Output()), _dom_classes=('widget-inte…

interactive(children=(IntSlider(value=82, description='SLICE', max=165), Output()), _dom_classes=('widget-inte…

interactive(children=(IntSlider(value=87, description='SLICE', max=175), Output()), _dom_classes=('widget-inte…

interactive(children=(IntSlider(value=82, description='SLICE', max=165), Output()), _dom_classes=('widget-inte…

**3.2** Use Bias Field Correction
- We will use N4 Bias Field Correction (SimpleITK) on the cropped image

#### Create head mask

In [34]:
inputImage = cropped_img_sitk

head_mask = sitk.RescaleIntensity(inputImage , 0, 255)
head_mask = sitk.LiThreshold(head_mask,0,1)

show_compare(inputImage,
            head_mask, 
            cmap='gray')

interactive(children=(IntSlider(value=87, description='SLICE', max=175), Output()), _dom_classes=('widget-inte…

interactive(children=(IntSlider(value=82, description='SLICE', max=165), Output()), _dom_classes=('widget-inte…

#### Bias Correction

In [35]:
shrinkFactor = 4

inputImage = sitk.Shrink(cropped_img_sitk, [ shrinkFactor ] * inputImage.GetDimension() )
maskImage = sitk.Shrink( head_mask, [ shrinkFactor ] * inputImage.GetDimension() )

bias_corrector = sitk.N4BiasFieldCorrectionImageFilter()

corrected = bias_corrector.Execute(inputImage, maskImage)

#### Get image corrected

In [36]:
log_bias_field = bias_corrector.GetLogBiasFieldAsImage(cropped_img_sitk)
corrected_image_full_resolution = cropped_img_sitk / sitk.Exp( log_bias_field )

show_compare(cropped_img_sitk,
            corrected_image_full_resolution, 
            cmap='gray')

interactive(children=(IntSlider(value=87, description='SLICE', max=175), Output()), _dom_classes=('widget-inte…

interactive(children=(IntSlider(value=82, description='SLICE', max=165), Output()), _dom_classes=('widget-inte…

#### Save the image

In [37]:
sitk.WriteImage(corrected_image_full_resolution, BASE_DIR+"/corrected-T1w.nii.gz")

**4.** Ants Registration of raw image (mobile) to atlas (fixed)

**Two command lines:**
- Registration of the corrected image **'corrected-T1w.nii.gz'** to the atlas **'atlas-T1w.nii.gz'** to get the output of the affine transform **'corrected_to_atlas_0GenericAffine.mat'** and the inverse displacement field **'corrected_to_atlas_1InverseWarp.nii.gz'** 
- Apply the reverse transform on the label image **'atlas-integer-labels.nii.gz'** to get **'label_to_corrected.nii.gz'**

In [38]:
#Create bash script
with open ('ants_registration.sh', 'w') as rsh:
    rsh.write('''\
#! /bin/bash
antsRegistrationSyNQuick.sh -d 3 -f atlas-T1w.nii.gz -m corrected-T1w.nii.gz -o corrected_to_atlas_ -n 4
antsApplyTransforms -d 3 -i atlas-integer-labels.nii.gz -r corrected-T1w.nii.gz -o label_to_corrected.nii.gz -t [corrected_to_atlas_0GenericAffine.mat, 1] -t corrected_to_atlas_1InverseWarp.nii.gz -n NearestNeighbor
''')

In [39]:
#Give access to bash script
process = subprocess.Popen(["chmod", "u+x", "ants_registration.sh"])
process.wait()

print("Completed!")

Completed!


In [40]:
#Execute bash script : 
#exit_code = subprocess.call('./ants_registration.sh')
#print(exit_code)

#Instead, use command line in terminal: ./ants_registration.sh

In [42]:
#Compare labels in different colors with the image of interest
img1_sitk = sitk.ReadImage(BASE_DIR+"/"+"corrected-T1w.nii.gz", sitk.sitkFloat32)
img2_sitk = sitk.ReadImage(BASE_DIR+"/"+"label_to_corrected.nii.gz", sitk.sitkFloat32)
show_compare(img1_sitk,img2_sitk,cmap='nipy_spectral')

interactive(children=(IntSlider(value=87, description='SLICE', max=175), Output()), _dom_classes=('widget-inte…

interactive(children=(IntSlider(value=82, description='SLICE', max=165), Output()), _dom_classes=('widget-inte…

In [45]:
#Superpose the merged labels on the corrected image
show_superpose("corrected-T1w.nii.gz","label_to_corrected.nii.gz",thickness=1)

interactive(children=(IntSlider(value=87, description='SLICE', max=175), Output()), _dom_classes=('widget-inte…

interactive(children=(IntSlider(value=82, description='SLICE', max=165), Output()), _dom_classes=('widget-inte…

**5.** Thresholding on integer labels and volume extraction

In [46]:
def process_label_im(im):
    """Compute statistics on an image using sitk.LabelShapeStatisticsImageFilter class.
    
    Argument : sitk image in integer format (sitkInt32 for example)

    Return the list of label indexes and their respective size.
    """

    labstats = sitk.LabelShapeStatisticsImageFilter()
    labstats.Execute(im)
    labels = list(labstats.GetLabels())
    sizes=[labstats.GetNumberOfPixels(i) for i in labels]

    return labels,sizes

In [47]:
im= sitk.ReadImage(BASE_DIR+"/label_to_corrected.nii.gz", sitk.sitkInt32)
labels,sizes=process_label_im(im)

In [48]:
#Create pandas dataframe
data = {'label': labels,
        'volume': sizes}

volume = pd.DataFrame(data)

In [49]:
#Merge the volume dataframe with the look up table from Freesurfer
result = pd.merge(table,volume,on="label")
print(result)

    label                           name  volume
0       1         Left-Cerebral-Exterior   12740
1       2     Left-Cerebral-White-Matter    1366
2       3           Left-Cerebral-Cortex    6999
3       4         Left-Lateral-Ventricle    9259
4       5              Left-Inf-Lat-Vent    9264
5       6       Left-Cerebellum-Exterior     373
6       7   Left-Cerebellum-White-Matter    1132
7       8         Left-Cerebellum-Cortex    1151
8       9                  Left-Thalamus     495
9      10          Left-Thalamus-Proper*     515
10     11                   Left-Caudate     625
11     12                   Left-Putamen     678
12     13                  Left-Pallidum     704
13     14                  3rd-Ventricle     658
14     15                  4th-Ventricle    1552
15     16                     Brain-Stem    1556
16     17               Left-Hippocampus    1916
17     18                  Left-Amygdala    1818
18     19                    Left-Insula    2403
19     20           

In [50]:
#Save into csv file
local_file="result_of_registration_BV.csv"
result.to_csv(local_file)

#Send it to Dropbox repository
dropbox_file_path="/recruitment_project/"+local_file
dropbox_upload_file("", local_file, dropbox_file_path);

# Improvement ideas

- Use automatic ITK-SkullStrip before Bias Field Correction to remove background noise and background artifacts (try to convert between itk and simpleITK)
- Create a pipeline
- Find a permanent method to access Dropbox